<a href="https://colab.research.google.com/github/clear21/Answer-Mashine/blob/master/RNN_jeopardy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
import urllib.request
import json
import pandas as pd
import numpy as np

from keras.layers import Dense , Activation , SimpleRNN , LSTM , Embedding
from keras.models import Sequential

from datetime import datetime

import re

In [0]:
#■関数

#◆jsonファイルで保存
#引数：保存対象オブジェクト、ファイル名(拡張子なし)
def save_as_json(obj , fname):
    json_fname_with_extension = fname + '.json'
    fw = open(json_fname_with_extension , 'w')
    json.dump(obj , fw , indent = 4)
    fw.close()
    files.download(json_fname_with_extension)

#◆jsonファイルを取得
#引数：対象ファイルのパス
def import_json(fpass):
    fr = open(fpass , 'r')
    f = json.load(fr)
    fr.close()
    return f

In [56]:
#QAデータセットの取得（JEOPARDY_CSV.csv）
files.upload()

# qa_data_url = r'https://drive.google.com/file/d/0BwT5wj_P7BKXUl9tOUJWYzVvUjA/view'
# qa_file_name = 'JEOPARDY_CSV.csv'
# urllib.request.urlretrieve(qa_data_url , qa_file_name)

Saving JEOPARDY_CSV.csv to JEOPARDY_CSV (1).csv


KeyboardInterrupt: ignored

In [75]:
#QAデータをデータフレーム化（Question と Answer を使用する）
qa_csv = r'JEOPARDY_CSV.csv'
qa_df = pd.read_csv(qa_csv)[[' Question' , ' Answer']]

#画像URLありの質問　＆　Answerが１wordでない　を除外
qa_df = qa_df[~qa_df[' Question'].str.contains('<a href=')]
print('画像URLありの質問' , '除外後' , len(qa_df))

qa_df[' Answer'] = qa_df[' Answer'].str.strip()
qa_df = qa_df[~qa_df[' Answer'].str.contains(' ' , na = True)]
print('Answerが１wordでない' , '除外後' , len(qa_df))
qa_df = qa_df.reset_index(drop = True)


#テストのため、行数削減
qa_df = qa_df[:2000]
qa_df = qa_df.reset_index(drop = True)

print('len(qa_df)' , len(qa_df))

画像URLありの質問 除外後 206407
Answerが１wordでない 除外後 89070
len(qa_df) 2000


In [76]:
#■文字
#関数：文字列→単価リスト（to_word_set～で使用）
#第二引数：「Q」＝Question用処理、「A」＝Answer用処理
def BK_sentence2wordlist(sentence , QorA):
    sentence = str(sentence)
    if QorA == 'Q':
        return list(map(lambda x : x.lower() , re.sub(r' +' , ' ' , sentence.replace(',' , ' , ').replace('.' , ' . ').replace('?' , ' ? ').replace('"' , '').replace('$' , ' $ ').replace('(' , ' ').replace(')' , ' '))))
    elif QorA == 'A':
        return sentence.lower()
      
#■単語
#関数：文字列→単価リスト（to_word_set～で使用）
#第二引数：「Q」＝Question用処理、「A」＝Answer用処理      
def sentence2wordlist(sentence , QorA):
    sentence = str(sentence)
    if QorA == 'Q':
        return list(map(lambda x : x.lower() , re.sub(r' +' , ' ' , sentence.replace(',' , ' , ').replace('.' , ' . ').replace('?' , ' ? ').replace('"' , '').replace('$' , ' $ ').replace('(' , ' ').replace(')' , ' ')).split(' ')))
    elif QorA == 'A':
        return sentence.lower()

#関数：文字列のリスト→単語の集合（重複無）※Question用
def to_word_set_Qustion(sentence_list):
    word_split_list = []
    for sentence in sentence_list:
        word_split_list += sentence2wordlist(sentence , 'Q')
    return set(word_split_list)

#関数：文字列のリスト→単語の集合（重複無）※Answer用
def to_word_set_Answer(sentence_list):
    word_split_list = []
    for sentence in sentence_list:    
        word_split_list.append(sentence2wordlist(sentence , 'A'))
    return set(word_split_list)
  
#単語辞書作成
#--単語一覧
q_word_set = to_word_set_Qustion(qa_df[' Question'])
a_word_set = to_word_set_Answer(qa_df[' Answer'])

# word_set = q_word_set | a_word_set
# del q_word_set , a_word_set

#Q用：word2id_q、id2word_q　　A用：word2id_a、id2word_a
#--Q用
word_list_q = sorted(list(q_word_set))
word2id_q = {word_list_q[i] : i for i in range(len(word_list_q))}
id2word_q = {i : word_list_q[i] for i in range(len(word_list_q))}

del q_word_set , word_list_q

#--A用
word_list_a = sorted(list(a_word_set))
word2id_a = {word_list_a[i] : i for i in range(len(word_list_a))}
id2word_a = {i : word_list_a[i] for i in range(len(word_list_a))}

del a_word_set , word_list_a

print('word2id_q' , len(word2id_q))
print('word2id_a' , len(word2id_a))

# word_list = sorted(list(word_set))
# word2id = {word_list[i] : i for i in range(len(word_list))}
# id2word = {i : word_list[i] for i in range(len(word_list))}
# del word_set , word_list

word2id_q 7630
word2id_a 1667


In [77]:
#モデルの型
HIDDEN_SIZE = 256
#NUM_ITERATIONS = 25
SEQLEN = 10
BATCH_SIZE = 128
NUM_EPOCHS_PER_ITERATION = 32
VOCAB_SIZE = len(word2id_q)
EMBED_SIZE = 128

model = Sequential()
model.add(Embedding(VOCAB_SIZE
                   , output_dim = EMBED_SIZE
                   , embeddings_initializer = 'glorot_uniform'
                   , input_length = SEQLEN))
# model.add(SimpleRNN(HIDDEN_SIZE 
#                     , return_sequences = False
#                     , stateful = False
#                     #, batch_input_shape = (BATCH_SIZE , None , len(word2id_q)))
#                     , input_shape = (None , len(word2id_q)))
#                    )
model.add(LSTM(HIDDEN_SIZE 
                     , return_sequences = False 
                     , input_shape = (None , len(word2id_q))))
model.add(Dense(len(word2id_a)))
model.add(Activation("softmax"))

model.compile(loss = "categorical_crossentropy" , optimizer = "rmsprop")

#入力値として、各Questionに対して、SEQLEN文字ずつ区切ったものを作成し、Answerをラベルとする。
one_fit_qa_num = len(qa_df)

word_split_question_list = []
word_split_answer_list = []

print(model.summary())

start = datetime.now()

#テスト
for test_no in range(1):
    for qa_no in range(len(qa_df)):

        word_split_question_list.append( sentence2wordlist(qa_df[' Question'][qa_no] , 'Q') )
        word_split_answer_list.append( sentence2wordlist(qa_df[' Answer'][qa_no] , 'A') )

        if True if one_fit_qa_num == 1 else (qa_no % (one_fit_qa_num - 1) == 0 and qa_no != 0) :
        #if ((0 if one_fit_qa_num == 1 else qa_no % (one_fit_qa_num - 1)) == 0 or qa_no == len(qa_df) - 1) and (True if one_fit_qa_num == 1 else qa_no != 0) :
            #入力用
            word_per_SEQLEN = []
            #ラベル
            y = []

            for word_split_question , word_split_answer in zip(word_split_question_list , word_split_answer_list):
                append_y_size = 0

                #入力
                if (len(word_split_question) - SEQLEN + 1) > 0:
                    for i in range(len(word_split_question) - SEQLEN + 1):
                        word_per_SEQLEN.append(word_split_question[i : i + SEQLEN])
                        append_y_size += 1
                else :
                    word_per_SEQLEN.append(word_split_question + ['' for i in range(SEQLEN - len(word_split_question))])
                    append_y_size += 1

                #ラベル
                y = y + [[1 if j == word2id_a[word_split_answer] else 0 for j in range(len(word2id_a))] for i in range(append_y_size)]

            #※idが要素
            X = np.zeros((len(word_per_SEQLEN) , SEQLEN) , dtype = np.int)
            
            #※idが要素
            for i in range(len(word_per_SEQLEN)):
                w_list = word_per_SEQLEN[i]
                for j in range(SEQLEN):
                    wid = word2id_q[w_list[j]]
                    X[i , j] = wid
                    
            print('X.shape' , np.array(X).shape)
            print('y.shape' , np.array(y).shape)
  
            #※one_hot作成
            #X = np.zeros((len(word_per_SEQLEN) , SEQLEN , len(word2id_q)) , dtype = np.bool)             
            
            #※one_hot作成
            #for i in range(len(word_per_SEQLEN)):
                #w_list = word_per_SEQLEN[i]
                #for j in range(SEQLEN):
                    #wid = word2id_q[w_list[j]]
                    #X[i , j , wid] = 1

            #×　ラベルの値
            #×　y = [[1 if j == word2id[word_split_answer] else 0 for j in range(len(word2id))] for i in range(len(word_per_SEQLEN))]  

            #学習
            model.fit(np.array(X) , np.array(y) , batch_size = BATCH_SIZE , epochs = NUM_EPOCHS_PER_ITERATION , shuffle = True)

            model.reset_states()            

            #初期化
            word_split_question_list = []
            word_split_answer_list = []

            #if qa_no == (one_fit_qa_num - 1):
                #print('X' , np.array(X).shape)
                #print('y' , len(y))
                #break
                
#1学習の時間
end = datetime.now()
print('learn time:' , end - start)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 10, 128)           976640    
_________________________________________________________________
lstm_12 (LSTM)               (None, 256)               394240    
_________________________________________________________________
dense_12 (Dense)             (None, 1667)              428419    
_________________________________________________________________
activation_12 (Activation)   (None, 1667)              0         
Total params: 1,799,299
Trainable params: 1,799,299
Non-trainable params: 0
_________________________________________________________________
None
X.shape (12637, 10)
y.shape (12637, 1667)
Epoch 1/32
12637/12637 [==============================] - 5s 409us/step - loss: 7.1359
Epoch 2/32
12637/12637 [==============================] - 3s 260us/step - loss: 6.6769
Epoch 3/32
12637/12637 [===================

In [50]:
#■関数

#※one_hot作成
#wordのリスト　→　入力用行列（1 × 入力word数 × len(word2id_q)）
#※各word が word2id_q に存在すること（＝絶対条件：小文字）
def BK_create_X(word_list , word2id_q = word2id_q):
    wid_list = [word2id_q[word] for word in word_list]
    return_X = np.zeros((1 , len(word_list) , len(word2id_q)) , dtype = np.bool)
    for j in range(len(wid_list)):
        return_X[0 , j , wid_list[j]] = 1
    return return_X

#※単語idが要素
#wordのリスト　→　入力用行列（1 × 入力word数 × len(word2id_q)）
#※各word が word2id_q に存在すること（＝絶対条件：小文字）
def create_X(word_list , word2id_q = word2id_q):
    wid_list = [word2id_q[word] for word in word_list]
    return_X = np.zeros((1 , len(word_list)) , dtype = int)
    for j in range(len(word_list)):
        return_X[0 , j] = wid_list[j]
    return return_X

#test_word_list = list('For the last 8 years of his life, Galileo was'.lower())
#test_word_list = ['f', 'o', 'r', ' ', 't', 'h', 'e', ' ', 'l', 'a', 's', 't', ' ', '8', ' ', 'y', 'e', 'a', 'r', 's', ' ', 'o', 'f', ' ', 'h', 'i', 's', ' ', 'l', 'i', 'f', 'e', ' ', ',', ' ', 'g', 'a', 'l', 'i', 'l', 'e', 'o', ' ', 'w', 'a', 's', ' ', 'u', 'n', 'd', 'e', 'r', ' ', 'h', 'o', 'u', 's', 'e', ' ', 'a', 'r', 'r', 'e', 's', 't', ' ', 'f', 'o', 'r', ' ', 'e', 's', 'p', 'o', 'u', 's', 'i', 'n', 'g', ' ', 't', 'h', 'i', 's', ' ', 'm', 'a', 'n', "'", 's', ' ', 't', 'h', 'e', 'o', 'r', 'y'] #copernicus
#test_word_list = ['in', '1963', ',', 'live', 'on', 'the', 'art', 'linkletter', 'show', ',', 'this', 'company', 'served', 'its', 'billionth', 'burger'] #mcdonald's
#test_word_list = ['for', 'the', 'last', '8', 'years', 'of', 'his', 'life', ',', 'galileo', 'was', 'under', 'house', 'arrest', 'for', 'espousing', 'this', "man's", 'theory'] #copernicus
#test_word_list = ['the', 'city', 'of', 'yuma', 'in', 'this', 'state', 'has', 'a', 'record', 'average', 'of', '4', ',', '055', 'hours', 'of', 'sunshine', 'each', 'year'] #arizona
test_word_list = ['a' , 'small' , 'demon' , ',' , 'or' , 'a' , 'mischievous' , 'child' , 'who' , 'might' , 'be' , 'a' , 'little' , 'demon'] #imp
#test_word_list = ['in' , 'the' , 'mid-18th' , 'c' , '.' , 'mikhail' , 'lomonosov' , 'became' , 'the' , 'first' , 'scientist' , 'to' , 'record' , 'the' , 'freezing' , 'of' , 'this' , 'liquid' , 'metal'] #mercury

print(test_word_list)

n = 3

test_X = create_X(test_word_list)
#print(test_X)
#print(len(test_X[0]))
test_X = np.array([[test_X[0][i] for i in range(len(test_X[0])) if i in [1,2,3,5,7,8,9,10,11,12] ]])
#test_X = np.array([[test_X[0][n : n + SEQLEN]]])

print(test_X)

# start_no = 0
# test_X = create_X(test_word_list[start_no : start_no + SEQLEN])

pred = model.predict(test_X , verbose = 0)[0]

['a', 'small', 'demon', ',', 'or', 'a', 'mischievous', 'child', 'who', 'might', 'be', 'a', 'little', 'demon']
[[ 431 6321 2132   18 4950  431 4512 1585 7435 4470  991  431 4116 2132]]
14
[[6321 2132   18  431 1585 7435 4470  991  431 4116]]


In [51]:
#Answerの候補
out_top_n = 10

enum_pred = list(enumerate(pred))
enum_pred.sort(key = lambda x : -x[1])

top_enum_pred = enum_pred[:out_top_n]
for i in range(len(top_enum_pred)):
    print((i+1) , '位' , id2word_a[top_enum_pred[i][0]] , top_enum_pred[i][1] )

print(id2word_a[np.argmax(pred)])

1 位 imp 0.998248
2 位 bananas 0.0006462039
3 位 souvenir 0.00016199931
4 位 lexicon 9.285988e-05
5 位 shofar 3.7878155e-05
6 位 zeus 2.7323573e-05
7 位 cocktail 2.3245078e-05
8 位 macrobiotic 2.2813223e-05
9 位 bread/dough 1.7220736e-05
10 位 sloth 1.720678e-05
imp


In [0]:
#embedの重みのダウンロード
# embed_layer = model.layers[0]
# embed_weights = embed_layer.get_weights()[0]

# pd.DataFrame(weights).to_csv('test.csv')
# files.download('test.csv')

In [0]:
#■モデルのダウンロード_1
#--重み
weights_file_name = 'weights_rnn_qa.hdf5'
model.save_weights(weights_file_name)

files.download(weights_file_name)

In [0]:
#■モデルのダウンロード_2
#--モデル構造
model_file_name = 'model_rnn_qa.json'
with open(model_file_name , 'w') as f:
    f.write(model.to_json())
    
files.download(model_file_name)

In [0]:
#■word2id_q、id2word_q、word2id_a、id2word_a のダウンロード
obj_dict = {'word2id_q':word2id_q , 'word2id_a':word2id_a , 'id2word_q':id2word_q , 'id2word_a':id2word_a}

for name , obj in obj_dict.items():
    save_as_json(obj , name)

In [0]:
qa_df.head(5)

In [0]:
id2word_a

In [0]:
#Answer が 1word　でない　を除外する前
print(len(word2id))

In [0]:
#Answer が 1word　でない　を除外した後
print(len(word2id))

In [0]:
print(word2id['last'])
print(word2id['8'])
print(word2id['years'])
print(word2id['of'])
print(word2id['his'])


print(word2id['galileo'])
print(word2id['was'])


In [0]:
n = len(qa_df) - 1

print(qa_df[' Question'][n])
print(qa_df[' Answer'][n])

In [0]:
a_word_set
#word2id